In [ ]:
# Video compressor w=200 (from ...\basedata to ...\basedata-w200)

import os
import subprocess
from pathlib import Path

# -----------------------------
# ffmpeg check
# -----------------------------
def check_ffmpeg():
    try:
        subprocess.run(
            ["ffmpeg", "-version"],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
    except FileNotFoundError:
        raise SystemExit("Error: ffmpeg.exe not found. Add ffmpeg to PATH.")

check_ffmpeg()

# -----------------------------
# Fixed source/destination paths
# -----------------------------
SRC = r"C:\Users\dart\Documents\GitHub\basedata\vids"
DST = r"C:\Users\dart\Documents\GitHub\basedata-w200\vids"

SRC = os.path.normpath(SRC)
DST = os.path.normpath(DST)

if not os.path.isdir(SRC):
    raise SystemExit(f"Source folder missing: {SRC}")

os.makedirs(DST, exist_ok=True)

print("SOURCE:", SRC)
print("DEST  :", DST)
print("Mode  : Auto-process missing files only")
print("--------------------------------------------------\n")

# -----------------------------
# Main logic
# -----------------------------
video_exts = {".mp4", ".mkv", ".webm"}
tasks = []

# Scan source
for root, dirs, files in os.walk(SRC):
    for file in files:
        ext = os.path.splitext(file)[1].lower()
        if ext not in video_exts:
            continue

        src_file = os.path.join(root, file)
        rel = os.path.relpath(src_file, SRC)
        dst_file = os.path.join(DST, rel)
        dst_dir = os.path.dirname(dst_file)

        # If missing in destination -> add to task list
        if not os.path.exists(dst_file):
            tasks.append((src_file, dst_file, dst_dir))

print(f"Total missing videos found: {len(tasks)}\n")

# -----------------------------
# Process missing videos
# -----------------------------
for src_file, dst_file, dst_dir in tasks:
    print(f"Processing: {src_file}")
    print(f"Output    : {dst_file}")

    os.makedirs(dst_dir, exist_ok=True)

    cmd = [
        "ffmpeg", "-y",
        "-i", src_file,
        "-vf", "scale=200:-2",
        "-c:v", "libx264",
        "-preset", "fast",
        "-crf", "28",
        "-c:a", "aac",
        "-b:a", "96k",
        dst_file
    ]

    subprocess.run(cmd)
    print()

print("--------------------------------------------------")
print("Processing complete. Missing files have been created.")


In [ ]:
# Image compressor w=200 (from ...\basedata to ...\basedata-w200)

import os
import subprocess
from pathlib import Path
from PIL import Image

# --------------------------------------------
# Check ImageMagick (optional but preferred)
# --------------------------------------------
def has_magick():
    try:
        subprocess.run(
            ["magick", "-version"],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        return True
    except FileNotFoundError:
        return False

USE_MAGICK = has_magick()

# --------------------------------------------
# Fixed Paths
# --------------------------------------------
SRC = r"C:\Users\dart\Documents\GitHub\basedata\imgs"
DST = r"C:\Users\dart\Documents\GitHub\basedata-w200\imgs"

SRC = os.path.normpath(SRC)
DST = os.path.normpath(DST)

if not os.path.isdir(SRC):
    raise SystemExit(f"Source folder missing: {SRC}")

os.makedirs(DST, exist_ok=True)

print("SOURCE:", SRC)
print("DEST  :", DST)
print("Mode  : Auto sync missing -> resize to 200w WEBP")
print("Using ImageMagick:", USE_MAGICK)
print("--------------------------------------------------\n")

# --------------------------------------------
# Allowed image types
# --------------------------------------------
img_exts = {".jpg", ".jpeg", ".png", ".webp"}

tasks = []

# --------------------------------------------
# Scan source
# --------------------------------------------
for root, dirs, files in os.walk(SRC):
    for file in files:
        ext = os.path.splitext(file)[1].lower()
        if ext not in img_exts:
            continue

        src_file = os.path.join(root, file)
        rel = os.path.relpath(src_file, SRC)

        dst_file = os.path.join(DST, os.path.splitext(rel)[0] + ".webp")  # force output .webp
        dst_dir = os.path.dirname(dst_file)

        if not os.path.exists(dst_file):
            tasks.append((src_file, dst_file, dst_dir))

print(f"Total missing images: {len(tasks)}\n")

# --------------------------------------------
# Process missing images
# --------------------------------------------
for src_file, dst_file, dst_dir in tasks:
    print(f"Processing: {src_file}")
    print(f"Output    : {dst_file}")

    os.makedirs(dst_dir, exist_ok=True)

    if USE_MAGICK:
        # Use ImageMagick for highest stability
        cmd = [
            "magick", src_file,
            "-resize", "200x",
            "-quality", "85",
            dst_file
        ]
        subprocess.run(cmd)
    else:
        # Pillow fallback
        img = Image.open(src_file)
        w, h = img.size
        new_h = int(h * (200 / w))
        img = img.resize((200, new_h))
        img.save(dst_file, "WEBP", quality=85)

    print()

print("--------------------------------------------------")
print("Processing complete. Missing images have been created.")


In [ ]:
# Webp Convertor auto destination 

import os
import subprocess

root = r"C:\Users\dart\Documents\GitHub\basedata\imgs"

if not os.path.isdir(root):
    raise SystemExit("Invalid path.")

allowed = {".webp"}

for dp, _, files in os.walk(root):
    for f in files:
        ext = os.path.splitext(f)[1].lower()
        full = os.path.join(dp, f)

        # skip WEBP
        if ext in allowed:
            continue

        name = os.path.splitext(f)[0]
        out_file = os.path.join(dp, f"{name}.webp")

        # convert
        subprocess.run(["cwebp", "-lossless", full, "-o", out_file], check=False)

        # delete only if output exists
        if os.path.exists(out_file):
            os.remove(full)
            print(f"Converted: {full} -> {out_file}")


In [ ]:
# Webp convertor custom path

import os
import subprocess

# ask for target path
target = input("Enter target folder path: ").strip()

if not os.path.isdir(target):
    raise SystemExit("Invalid path.")

exts = {".jpg", ".jpeg", ".png", ".gif"}

for f in os.listdir(target):
    full = os.path.join(target, f)
    name, ext = os.path.splitext(f)

    if ext.lower() not in exts:
        continue

    out_file = os.path.join(target, f"{name}.webp")

    subprocess.run(["cwebp", "-lossless", full, "-o", out_file], check=False)

    if os.path.exists(out_file):
        os.remove(full)


In [ ]:
# Webp Checker 

import os

root = r"C:\Users\dart\Documents\GitHub\basedata-w200\imgs"

if not os.path.isdir(root):
    raise SystemExit("Invalid path.")

allowed = {".webp"}

print("Non-WEBP files:\n")

for dp, _, files in os.walk(root):
    for f in files:
        ext = os.path.splitext(f)[1].lower()
        if ext not in allowed:
            rel = os.path.relpath(os.path.join(dp, f), root)
            print(rel)
